<a href="https://colab.research.google.com/github/FrancLis/Multivariate-Time-Series-Forecasting/blob/main/ARIMA_ARIMAX_SARIMA_SARIMAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pmdarima
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from sklearn.metrics import max_error
from sklearn.metrics import r2_score

In [ ]:
!pip install termocolor
from termcolor import colored

print(colored('hello', 'red'), colored('world', 'green'))

In [ ]:
df = pd.read_csv('/content/PG.csv', parse_dates=['Date'] ,index_col='Date')

In [ ]:
df

# Overview of the data

# Train & Test Data split

In [ ]:
#split data into train and training set
train_data, test_data = df[:-1315], df[-1315:]

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
#split data into train and training set
plt.figure(figsize=(10,6))
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.plot(train_data.Close, 'green', label='Train data')
plt.plot(test_data.Close, 'blue', label='Test data')
plt.legend()

In [ ]:
train_data_endog = train_data.Close.values
test_data_endog = test_data.Close.values

In [ ]:
train_data_endog = train_data_endog.reshape(-1, 1)
test_data_endog = test_data_endog.reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import PowerTransformer

pt_endog = PowerTransformer(method='box-cox')
train_data_endog = pt_endog.fit_transform(train_data_endog)
test_data_endog = pt_endog.transform(test_data_endog)

In [ ]:
model_autoARIMA = auto_arima(train_data_endog , start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

In [ ]:
#Summary of the model
model_autoARIMA.summary()

In [ ]:
model_autoARIMA.plot_diagnostics(figsize=(10,8))
plt.show()

The residual plots for the auto ARIMA model look pretty good.
Histogram plus estimated density plot: The red KDE line follows closely with the N(0,1) line. This is a good indication that the residuals are normally distributed.
The Q-Q-plot: Shows that the ordered distribution of residuals (blue dots) follows the linear trend of the samples taken from a standard normal distribution with N(0, 1). This is an indication that the residuals are normally distributed.
The standardize residual plot: The residuals over time don’t display any obvious seasonality and appear to be white noise.
The Correlogram plot: Shows that the time series residuals have low correlation with lagged versions of itself.
Our model is not perfect yet & It needs a few more tweaks.

So how to interpret the plot diagnostics?

Top left: The residual errors seem to fluctuate around a mean of zero and have a uniform variance.

Top Right: The density plot suggest normal distribution with mean zero.

Bottom left: All the dots should fall perfectly in line with the red line. Any significant deviations would imply the distribution is skewed.

Bottom Right: The Correlogram, aka, ACF plot shows the residual errors are not autocorrelated. Any autocorrelation would imply that there is some pattern in the residual errors which are not explained in the model. So you will need to look for more X’s (predictors) to the model.

In [ ]:
prediction_ARIMA = pd.DataFrame(model_autoARIMA.predict(n_periods = 1315), index=test_data.index)
prediction_ARIMA.columns = ['Predictions_ARIMA']
prediction_rescaled_ARIMA = pt_endog.inverse_transform(prediction_ARIMA.values)
prediction_ARIMA['Predictions_scaled_ARIMA'] = prediction_rescaled_ARIMA
prediction_ARIMA

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train_data.Close ,label="Training")
plt.plot(test_data.Close ,label="Test")
plt.plot(prediction_ARIMA.Predictions_scaled_ARIMA, label="Predicted_Arima")
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
def evaluate_prediction(predicted, actual, model_name):
        mse = mean_squared_error(predicted, actual)
        rsme = np.sqrt((mean_squared_error(predicted, actual)))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        max_err = max_error(actual, predicted)
        print(colored(model_name + ' performance:', 'red'))
        print('R^2: {:.4f} %'.format(r2 * 100))
        print('Mean Absolute Error: {:.4f}'.format(mae))
        print('Mean Squared Error: {:.4f}'.format(mse))
        print('Root Mean Squared Error: {:.4f}'.format(rsme))
        print('Max_error: {:.4f}'.format(max_err))
        print('')
        return

evaluate_prediction(prediction_ARIMA.Predictions_scaled_ARIMA, test_data.Close, 'ARIMA')

# ARIMAX

In [ ]:
exogenous_features = ["Open", "High", "Low", "Adj Close",
                      "Volume"]

In [ ]:
train_data_exog = train_data[exogenous_features]
test_data_exog = test_data[exogenous_features]

In [ ]:
from sklearn.preprocessing import PowerTransformer

pt_exog = PowerTransformer(method='box-cox')
train_data_exog = pt_exog.fit_transform(train_data_exog)
test_data_exog = pt_exog.transform(test_data_exog)

In [ ]:
model_autoARIMAX = auto_arima(train_data_endog , exogenous=train_data_exog, start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

In [ ]:
#Summary of the model
model_autoARIMAX.summary()

In [ ]:
model_autoARIMAX.plot_diagnostics(figsize=(10,8))
plt.show()

In [ ]:
prediction_ARIMAX = pd.DataFrame(model_autoARIMAX.predict(n_periods = 1315, exogenous=test_data_exog), index=test_data.index)
prediction_ARIMAX.columns = ['Predictions_ARIMAX']
prediction_rescaled_ARIMAX = pt_endog.inverse_transform(prediction_ARIMAX.values)
prediction_ARIMAX['Predictions_scaled_ARIMAX'] = prediction_rescaled_ARIMAX
prediction_ARIMAX

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train_data.Close ,label="Training")
plt.plot(test_data.Close ,label="Test")
plt.plot(prediction_ARIMAX.Predictions_scaled_ARIMAX, label="Predicted_Arimax")
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
def evaluate_prediction(predicted, actual, model_name):
        mse = mean_squared_error(predicted, actual)
        rsme = np.sqrt((mean_squared_error(predicted, actual)))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        max_err = max_error(actual, predicted)
        print(colored(model_name + ' performance:', 'red'))
        print('R^2: {:.4f} %'.format(r2 * 100))
        print('Mean Absolute Error: {:.4f}'.format(mae))
        print('Mean Squared Error: {:.4f}'.format(mse))
        print('Root Mean Squared Error: {:.4f}'.format(rsme))
        print('Max_error: {:.4f}'.format(max_err))
        print('')
        return

evaluate_prediction(prediction_ARIMAX.Predictions_scaled_ARIMAX, test_data.Close, 'ARIMAX')

# SARIMA

each models have parameters.

ARMA model:(p,q)

ARIMA model:(p,d,q)

SARIMA model:(p,d,q)(sp,sd,sq,s)

ARIMAX model:(p,d,q) + exog

SARIMAX model:(p,d,q)(sp,sd,sq,s) +exog

In [ ]:
model_autoSARIMA = auto_arima(train_data_endog , start_p=0, start_q=0,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=2, max_q=2, # maximum p and q
                      m=12,              # frequency of series
                      d=1,             
                      seasonal=True,  
                      start_P=0,
                      D=1,
                      start_Q=0, 
                      max_P=1,
                      max_Q=1,
                      trace=True,
                      max_order = 6,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

In [ ]:
#Summary of the model
model_autoSARIMA.summary()

In [ ]:
model_autoSARIMA.plot_diagnostics(figsize=(10,8))
plt.show()

In [ ]:
prediction_SARIMA = pd.DataFrame(model_autoSARIMA.predict(n_periods = 1315), index=test_data.index)
prediction_SARIMA.columns = ['Predictions_SARIMA']
prediction_rescaled_SARIMA = pt_endog.inverse_transform(prediction_SARIMA.values)
prediction_SARIMA['Predictions_scaled_SARIMA'] = prediction_rescaled_SARIMA
prediction_SARIMA

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train_data.Close ,label="Training")
plt.plot(test_data.Close ,label="Test")
plt.plot(prediction_SARIMA.Predictions_scaled_SARIMA, label="Predicted_Sarima")
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
def evaluate_prediction(predicted, actual, model_name):
        mse = mean_squared_error(predicted, actual)
        rsme = np.sqrt((mean_squared_error(predicted, actual)))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        max_err = max_error(actual, predicted)
        print(colored(model_name + ' performance:', 'red'))
        print('R^2: {:.4f} %'.format(r2 * 100))
        print('Mean Absolute Error: {:.4f}'.format(mae))
        print('Mean Squared Error: {:.4f}'.format(mse))
        print('Root Mean Squared Error: {:.4f}'.format(rsme))
        print('Max_error: {:.4f}'.format(max_err))
        print('')
        return

evaluate_prediction(prediction_SARIMA.Predictions_scaled_SARIMA, test_data.Close, 'SARIMA')

# SARIMAX

In [ ]:
model_autoSARIMAX = auto_arima(train_data_endog, exogenous=train_data_exog, start_p=0, start_q=0,
 max_p=2, max_q=2, # maximum p and q
                      m=12,              # frequency of series
                      d=1,             
                      seasonal=True,  
                      start_P=0, 
                      start_Q=0, 
                      max_P=2, 
                      max_Q=2,
                      trace=True,
                      max_order = 6,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

In [ ]:
#Summary of the model
model_autoSARIMAX.summary()

In [ ]:
model_autoSARIMAX.plot_diagnostics(figsize=(10,8))
plt.show()

In [ ]:
prediction_SARIMAX = pd.DataFrame(model_autoSARIMAX.predict(n_periods = 1315, exogenous=test_data_exog), index=test_data.index)
prediction_SARIMAX.columns = ['Predictions_SARIMAX']
prediction_rescaled_SARIMAX = pt_endog.inverse_transform(prediction_SARIMAX.values)
prediction_SARIMAX['Predictions_scaled_SARIMAX'] = prediction_rescaled_SARIMAX
prediction_SARIMAX

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train_data.Close ,label="Training")
plt.plot(test_data.Close ,label="Test")
plt.plot(prediction_SARIMAX.Predictions_scaled_SARIMAX, label="Predicted_Sarimax")
plt.legend(loc = 'upper left')
plt.show()

In [ ]:
def evaluate_prediction(predicted, actual, model_name):
        mse = mean_squared_error(predicted, actual)
        rsme = np.sqrt((mean_squared_error(predicted, actual)))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)
        max_err = max_error(actual, predicted)
        print(colored(model_name + ' performance:', 'red'))
        print('R^2: {:.4f} %'.format(r2 * 100))
        print('Mean Absolute Error: {:.4f}'.format(mae))
        print('Mean Squared Error: {:.4f}'.format(mse))
        print('Root Mean Squared Error: {:.4f}'.format(rsme))
        print('Max_error: {:.4f}'.format(max_err))
        print('')
        return

evaluate_prediction(prediction_SARIMAX.Predictions_scaled_SARIMAX, test_data.Close, 'SARIMAX')

I don't know the best way to estimate seasonal_order(sp,sd,sq,s) parameters.

parameter s:

1 for yearly
4 for quarterly
12 for monthly
52 for weekly
365 for daily

https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html

When we choose period 365,It will run out of memory.
It will probably, SARIMA model is unsuitable to solve this problem.
Forecasting with long seasonal periods(for R)
Deciding the value of period in seasonal ARIMA (for R)

For now, we choose period 1.

arima_model =  auto_arima(df_log, start_p=0, d=1, start_q=0, 
                          max_p=5, max_d=5, max_q=5, start_P=0, 
                          D=1, start_Q=0, max_P=5, max_D=5,
                          max_Q=5, m=12, seasonal=True, 
                          error_action='warn', trace = True,
                          supress_warnings=True, stepwise = True,
                          random = True,
                          random_state=20, n_fits=5)
